In [264]:
#pip install category_encoders

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from itertools import cycle, islice
import datetime as datetime
import glob
from matplotlib import pyplot
#import pycountry
#import plotly.express as px
#import psutil
#import kaleido
#import kora.install.orca
import plotly.graph_objects as go
import plotly.io as pio
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import log_loss, make_scorer
import category_encoders as ce

%matplotlib inline

plt.style.use('default') # haciendo los graficos un poco mas bonitos en matplotlib
#plt.rcParams['figure.figsize'] = (20, 10)

sns.set(style="whitegrid") # seteando tipo de grid en seaborn

pd.options.display.float_format = '{:20,.2f}'.format # suprimimos la notacion cientifica en los outputs

#import warnings
#warnings.filterwarnings('ignore')

plt.rcParams['figure.figsize'] = (15, 5)

In [2]:
def oneHotEncoding(df, columna):
    one_hot = pd.get_dummies(df[columna])
    df.drop(columna, axis=1, inplace=True)
    df = pd.concat([df, one_hot], axis=1)
    return df

# Train

In [3]:
train = pd.read_csv('Train_TP2_Datos_2020-2C.csv')

In [4]:
#Ignoro los otros estados ya que hay muy pocos de ellos y no se sacarles utilidad.
#Y convierto las victorias en 1 y derrotas en 0
train = train[(train.Stage == 'Closed Won') | (train.Stage == 'Closed Lost')]
train['Stage'] = train['Stage'].transform(func=lambda x : 1 if x=='Closed Won' else 0)

---

In [5]:
train['Bureaucratic_Code_0_Approval'] = train['Bureaucratic_Code_0_Approval'].transform(func=lambda x : 'Necesita' if x==1 else 'NoNecesita')
train['Bureaucratic_Code_0_Approved'] = train['Bureaucratic_Code_0_Approved'].transform(func=lambda x : 'Obtuvo' if x==1 else 'NoObtuvo')
train['Codigo_Burocratico'] = train['Bureaucratic_Code_0_Approval']+train['Bureaucratic_Code_0_Approved']

---

In [6]:
train['Pricing, Delivery_Terms_Quote_Appr'] = train['Pricing, Delivery_Terms_Quote_Appr'].transform(func=lambda x : 'Necesita' if x==1 else 'NoNecesita')
train['Pricing, Delivery_Terms_Approved'] = train['Pricing, Delivery_Terms_Approved'].transform(func=lambda x : 'Obtuvo' if x==1 else 'NoObtuvo')
train['Aprobacion_Especial'] = train['Pricing, Delivery_Terms_Quote_Appr']+train['Pricing, Delivery_Terms_Approved']

---

In [7]:
train.drop(train[train.Total_Amount.isnull()].index, inplace = True)

In [8]:
train = train.drop('ID', axis=1)
#train = train.drop('Territory', axis=1)
train = train.drop('Pricing, Delivery_Terms_Quote_Appr', axis=1)
train = train.drop('Pricing, Delivery_Terms_Approved', axis=1)
train = train.drop('Bureaucratic_Code_0_Approval', axis=1)
train = train.drop('Bureaucratic_Code_0_Approved', axis=1)
train = train.drop('Account_Created_Date', axis=1)
#train = train.drop('Source ', axis=1)
train = train.drop('Billing_Country', axis=1)
#train = train.drop('Account_Name', axis=1)
#train = train.drop('Opportunity_Name', axis=1)
#train = train.drop('Sales_Contract_No', axis=1)
train = train.drop('Account_Owner', axis=1)
#train = train.drop('Opportunity_Owner', axis=1)
train = train.drop('Account_Type', axis=1)
train = train.drop('Opportunity_Type', axis=1)
train = train.drop('Quote_Type', axis=1)
train = train.drop('Delivery_Terms', axis=1)
train = train.drop('Opportunity_Created_Date', axis=1)
train = train.drop('Brand', axis=1)
train = train.drop('Product_Type', axis=1)
train = train.drop('Size', axis=1)
train = train.drop('Product_Category_B', axis=1)
train = train.drop('Price', axis=1)
train = train.drop('Currency', axis=1)
train = train.drop('Last_Activity', axis=1)
train = train.drop('Quote_Expiry_Date', axis=1)
train = train.drop('Last_Modified_Date', axis=1)
train = train.drop('Last_Modified_By', axis=1)
train = train.drop('Product_Family', axis=1)
train = train.drop('Product_Name', axis=1)
#train = train.drop('ASP_Currency', axis=1)
#train = train.drop('ASP', axis=1)
train = train.drop('ASP_(converted)_Currency', axis=1)
#train = train.drop('ASP_(converted)', axis=1)
train = train.drop('Planned_Delivery_Start_Date', axis=1)
train = train.drop('Planned_Delivery_End_Date', axis=1)
train = train.drop('Month', axis=1)
train = train.drop('Delivery_Quarter', axis=1)
train = train.drop('Delivery_Year', axis=1)
train = train.drop('Actual_Delivery_Date', axis=1)
#train = train.drop('Total_Amount_Currency', axis=1)
#train = train.drop('Total_Amount', axis=1)
#train = train.drop('Total_Taxable_Amount_Currency', axis=1)
#train = train.drop('Total_Taxable_Amount', axis=1)
train = train.drop('Prod_Category_A', axis=1)
train = train.drop('Submitted_for_Approval', axis=1)
train = train.drop('Bureaucratic_Code', axis=1)
train = train.drop('Region', axis=1)

---

In [9]:
#train['TRF_totales'] = train.groupby('Opportunity_ID')['TRF'].transform('sum')
train['TRF_promedio'] = train.groupby('Opportunity_ID')['TRF'].transform('mean')
train = train.drop('TRF', axis=1)

---

In [10]:
#Completo los datos faltantes de 'Total_Taxable_Amount' con los de 'Total_Amount'
train['Total_Taxable_Amount'].loc[train.Total_Taxable_Amount==0.00] = train.loc[train.Total_Taxable_Amount==0.00]\
                                            .groupby(['Opportunity_Name'])['Total_Amount'].transform('sum')

train = train.drop('Opportunity_Name', axis=1)

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [11]:
toEUR = train[train.ASP_Currency=='EUR'].head(1)[['ASP_(converted)']].iloc[0,0]/train[train.ASP_Currency=='EUR'].head(1)[['ASP']].iloc[0,0]
toAUD = train[train.ASP_Currency=='AUD'].head(1)[['ASP_(converted)']].iloc[0,0]/train[train.ASP_Currency=='AUD'].head(1)[['ASP']].iloc[0,0]
toGBP = train[train.ASP_Currency=='GBP'].head(1)[['ASP_(converted)']].iloc[0,0]/train[train.ASP_Currency=='GBP'].head(1)[['ASP']].iloc[0,0]
toJPY = train[['ASP_(converted)']].iloc[12,0]/train[['ASP']].iloc[12,0]
#En el head(1) de JPY justo habia un nan. Ese numero magico 12 corresponde a la primera fila de moneda JPY con un valor valido

train = train.drop('ASP_Currency', axis=1)
train = train.drop('ASP', axis=1)
train = train.drop('ASP_(converted)', axis=1)

In [12]:
train['Total_Amount_(converted)']=0.0
train['Total_Amount_(converted)'].loc[train['Total_Amount_Currency']=='EUR'] = train['Total_Amount'].apply(lambda x: x*toEUR)
train['Total_Amount_(converted)'].loc[train['Total_Amount_Currency']=='JPY'] = train['Total_Amount'].apply(lambda x: x*toJPY)
train['Total_Amount_(converted)'].loc[train['Total_Amount_Currency']=='AUD'] = train['Total_Amount'].apply(lambda x: x*toAUD)
train['Total_Amount_(converted)'].loc[train['Total_Amount_Currency']=='GBP'] = train['Total_Amount'].apply(lambda x: x*toGBP)
train['Total_Amount_(converted)'].loc[train['Total_Amount_Currency']=='USD'] = train['Total_Amount'].apply(lambda x: x)

train['Total_Taxable_Amount_(converted)']=0.0
train['Total_Taxable_Amount_(converted)'].loc[train['Total_Taxable_Amount_Currency']=='EUR'] = train['Total_Taxable_Amount'].apply(lambda x: x*toEUR)
train['Total_Taxable_Amount_(converted)'].loc[train['Total_Taxable_Amount_Currency']=='JPY'] = train['Total_Taxable_Amount'].apply(lambda x: x*toJPY)
train['Total_Taxable_Amount_(converted)'].loc[train['Total_Taxable_Amount_Currency']=='AUD'] = train['Total_Taxable_Amount'].apply(lambda x: x*toAUD)
train['Total_Taxable_Amount_(converted)'].loc[train['Total_Taxable_Amount_Currency']=='GBP'] = train['Total_Taxable_Amount'].apply(lambda x: x*toGBP)
train['Total_Taxable_Amount_(converted)'].loc[train['Total_Taxable_Amount_Currency']=='USD'] = train['Total_Taxable_Amount'].apply(lambda x: x)

train = train.drop('Total_Amount_Currency', axis=1)
train = train.drop('Total_Amount', axis=1)
train = train.drop('Total_Taxable_Amount_Currency', axis=1)
train = train.drop('Total_Taxable_Amount', axis=1)

In [13]:
train['TA_promedio'] = train.groupby('Opportunity_ID')['Total_Amount_(converted)'].transform('mean')
train = train.drop('Total_Amount_(converted)', axis=1)

---

In [14]:
target_enc_terr = ce.TargetEncoder(train['Territory'])

# Fit the encoder using the categorical features and target
target_enc_terr.fit(train['Territory'], train['Stage'])

# Transform the features, rename the columns with _target suffix, and join to dataframe
train = train.join(target_enc_terr.transform(train['Territory']).add_suffix('_target'))

train = train.drop('Territory', axis=1)

---

In [15]:
target_enc_owner = ce.TargetEncoder(train['Opportunity_Owner'])

# Fit the encoder using the categorical features and target
target_enc_owner.fit(train['Opportunity_Owner'], train['Stage'])

# Transform the features, rename the columns with _target suffix, and join to dataframe
train = train.join(target_enc_owner.transform(train['Opportunity_Owner']).add_suffix('_target'))

train = train.drop('Opportunity_Owner', axis=1)

---

In [16]:
target_enc_source = ce.TargetEncoder(train['Source '])

# Fit the encoder using the categorical features and target
target_enc_source.fit(train['Source '], train['Stage'])

# Transform the features, rename the columns with _target suffix, and join to dataframe
train = train.join(target_enc_source.transform(train['Source ']).add_suffix('_target'))

train = train.drop('Source ', axis=1)

---

In [17]:
trainOp = train.drop_duplicates('Opportunity_ID')
trainOp = trainOp.drop('Opportunity_ID', axis=1)

---

In [18]:
target_enc_account_name = ce.TargetEncoder(trainOp['Account_Name'])

# Fit the encoder using the categorical features and target
target_enc_account_name.fit(trainOp['Account_Name'], trainOp['Stage'])

# Transform the features, rename the columns with _target suffix, and join to dataframe
trainOp = trainOp.join(target_enc_account_name.transform(trainOp['Account_Name']).add_suffix('_target'))

trainOp = trainOp.drop('Account_Name', axis=1)

---

In [19]:
target_enc_SCN = ce.TargetEncoder(trainOp['Sales_Contract_No'])

# Fit the encoder using the categorical features and target
target_enc_SCN.fit(trainOp['Sales_Contract_No'], trainOp['Stage'])

# Transform the features, rename the columns with _target suffix, and join to dataframe
trainOp = trainOp.join(target_enc_SCN.transform(trainOp['Sales_Contract_No']).add_suffix('_target'))

trainOp = trainOp.drop('Sales_Contract_No', axis=1)

---

In [20]:
target_enc_CB = ce.TargetEncoder(trainOp['Codigo_Burocratico'])

# Fit the encoder using the categorical features and target
target_enc_CB.fit(trainOp['Codigo_Burocratico'], trainOp['Stage'])

# Transform the features, rename the columns with _target suffix, and join to dataframe
trainOp = trainOp.join(target_enc_CB.transform(trainOp['Codigo_Burocratico']).add_suffix('_target'))

trainOp = trainOp.drop('Codigo_Burocratico', axis=1)

---

In [21]:
target_enc_AE = ce.TargetEncoder(trainOp['Aprobacion_Especial'])

# Fit the encoder using the categorical features and target
target_enc_AE.fit(trainOp['Aprobacion_Especial'], trainOp['Stage'])

# Transform the features, rename the columns with _target suffix, and join to dataframe
trainOp = trainOp.join(target_enc_AE.transform(trainOp['Aprobacion_Especial']).add_suffix('_target'))

trainOp = trainOp.drop('Aprobacion_Especial', axis=1)

---

In [22]:
train_label = []
train_data = []

train_label = trainOp["Stage"]
train_data = trainOp.iloc[:, trainOp.columns != 'Stage']

# Test

In [36]:
test = pd.read_csv('Test_TP2_Datos_2020-2C.csv')

---

In [37]:
test['Bureaucratic_Code_0_Approval'] = test['Bureaucratic_Code_0_Approval'].transform(func=lambda x : 'Necesita' if x==1 else 'NoNecesita')
test['Bureaucratic_Code_0_Approved'] = test['Bureaucratic_Code_0_Approved'].transform(func=lambda x : 'Obtuvo' if x==1 else 'NoObtuvo')
test['Codigo_Burocratico'] = test['Bureaucratic_Code_0_Approval']+test['Bureaucratic_Code_0_Approved']

---

In [38]:
test['Pricing, Delivery_Terms_Quote_Appr'] = test['Pricing, Delivery_Terms_Quote_Appr'].transform(func=lambda x : 'Necesita' if x==1 else 'NoNecesita')
test['Pricing, Delivery_Terms_Approved'] = test['Pricing, Delivery_Terms_Approved'].transform(func=lambda x : 'Obtuvo' if x==1 else 'NoObtuvo')
test['Aprobacion_Especial'] = test['Pricing, Delivery_Terms_Quote_Appr']+test['Pricing, Delivery_Terms_Approved']

---

In [39]:
test = test.drop('ID', axis=1)
#test = test.drop('Territory', axis=1)
test = test.drop('Pricing, Delivery_Terms_Quote_Appr', axis=1)
test = test.drop('Pricing, Delivery_Terms_Approved', axis=1)
test = test.drop('Bureaucratic_Code_0_Approval', axis=1)
test = test.drop('Bureaucratic_Code_0_Approved', axis=1)
test = test.drop('Account_Created_Date', axis=1)
#test = test.drop('Source ', axis=1)
test = test.drop('Billing_Country', axis=1)
#test = test.drop('Account_Name', axis=1)
#test = test.drop('Opportunity_Name', axis=1)
#test = test.drop('Sales_Contract_No', axis=1)
test = test.drop('Account_Owner', axis=1)
#test = test.drop('Opportunity_Owner', axis=1)
test = test.drop('Account_Type', axis=1)
test = test.drop('Opportunity_Type', axis=1)
test = test.drop('Quote_Type', axis=1)
test = test.drop('Delivery_Terms', axis=1)
test = test.drop('Opportunity_Created_Date', axis=1)
test = test.drop('Brand', axis=1)
test = test.drop('Product_Type', axis=1)
test = test.drop('Size', axis=1)
test = test.drop('Product_Category_B', axis=1)
test = test.drop('Price', axis=1)
test = test.drop('Currency', axis=1)
test = test.drop('Last_Activity', axis=1)
test = test.drop('Quote_Expiry_Date', axis=1)
test = test.drop('Last_Modified_Date', axis=1)
test = test.drop('Last_Modified_By', axis=1)
test = test.drop('Product_Family', axis=1)
test = test.drop('Product_Name', axis=1)
test = test.drop('ASP_Currency', axis=1)
test = test.drop('ASP', axis=1)
test = test.drop('ASP_(converted)_Currency', axis=1)
test = test.drop('ASP_(converted)', axis=1)
test = test.drop('Planned_Delivery_Start_Date', axis=1)
test = test.drop('Planned_Delivery_End_Date', axis=1)
test = test.drop('Month', axis=1)
test = test.drop('Delivery_Quarter', axis=1)
test = test.drop('Delivery_Year', axis=1)
test = test.drop('Actual_Delivery_Date', axis=1)
#test = test.drop('Total_Amount_Currency', axis=1)
#test = test.drop('Total_Amount', axis=1)
#test = test.drop('Total_Taxable_Amount_Currency', axis=1)
#test = test.drop('Total_Taxable_Amount', axis=1)
test = test.drop('Prod_Category_A', axis=1)
test = test.drop('Submitted_for_Approval', axis=1)
test = test.drop('Bureaucratic_Code', axis=1)
test = test.drop('Region', axis=1)

---

In [40]:
test['TRF_promedio'] = test.groupby('Opportunity_ID')['TRF'].transform('mean')
test = test.drop('TRF', axis=1)

---

In [41]:
test['Total_Taxable_Amount'].loc[test.Total_Taxable_Amount==0.00] = test.loc[test.Total_Taxable_Amount==0.00]\
                                            .groupby(['Opportunity_Name'])['Total_Amount'].transform('sum')

test = test.drop('Opportunity_Name', axis=1)

In [42]:
test['Total_Amount_(converted)']=0.0
test['Total_Amount_(converted)'].loc[test['Total_Amount_Currency']=='EUR'] = test['Total_Amount'].apply(lambda x: x*toEUR)
test['Total_Amount_(converted)'].loc[test['Total_Amount_Currency']=='JPY'] = test['Total_Amount'].apply(lambda x: x*toJPY)
test['Total_Amount_(converted)'].loc[test['Total_Amount_Currency']=='AUD'] = test['Total_Amount'].apply(lambda x: x*toAUD)
test['Total_Amount_(converted)'].loc[test['Total_Amount_Currency']=='GBP'] = test['Total_Amount'].apply(lambda x: x*toGBP)
test['Total_Amount_(converted)'].loc[test['Total_Amount_Currency']=='USD'] = test['Total_Amount'].apply(lambda x: x)

test['Total_Taxable_Amount_(converted)']=0.0
test['Total_Taxable_Amount_(converted)'].loc[test['Total_Taxable_Amount_Currency']=='EUR'] = test['Total_Taxable_Amount'].apply(lambda x: x*toEUR)
test['Total_Taxable_Amount_(converted)'].loc[test['Total_Taxable_Amount_Currency']=='JPY'] = test['Total_Taxable_Amount'].apply(lambda x: x*toJPY)
test['Total_Taxable_Amount_(converted)'].loc[test['Total_Taxable_Amount_Currency']=='AUD'] = test['Total_Taxable_Amount'].apply(lambda x: x*toAUD)
test['Total_Taxable_Amount_(converted)'].loc[test['Total_Taxable_Amount_Currency']=='GBP'] = test['Total_Taxable_Amount'].apply(lambda x: x*toGBP)
test['Total_Taxable_Amount_(converted)'].loc[test['Total_Taxable_Amount_Currency']=='USD'] = test['Total_Taxable_Amount'].apply(lambda x: x)

test = test.drop('Total_Amount_Currency', axis=1)
test = test.drop('Total_Amount', axis=1)
test = test.drop('Total_Taxable_Amount_Currency', axis=1)
test = test.drop('Total_Taxable_Amount', axis=1)

In [43]:
test['TA_promedio'] = test.groupby('Opportunity_ID')['Total_Amount_(converted)'].transform('mean')
test = test.drop('Total_Amount_(converted)', axis=1)

---

In [44]:
test = test.join(target_enc_terr.transform(test['Territory']).add_suffix('_target'))
test = test.drop('Territory', axis=1)

---

In [45]:
test = test.join(target_enc_owner.transform(test['Opportunity_Owner']).add_suffix('_target'))
test = test.drop('Opportunity_Owner', axis=1)

---

In [46]:
test = test.join(target_enc_source.transform(test['Source ']).add_suffix('_target'))
test = test.drop('Source ', axis=1)

---

In [47]:
testOp = test.drop_duplicates('Opportunity_ID')
IDs = testOp[['Opportunity_ID']].copy()
testOp = testOp.drop('Opportunity_ID', axis=1)

---

In [48]:
testOp = testOp.join(target_enc_account_name.transform(testOp['Account_Name']).add_suffix('_target'))
testOp = testOp.drop('Account_Name', axis=1)

---

In [49]:
testOp = testOp.join(target_enc_SCN.transform(testOp['Sales_Contract_No']).add_suffix('_target'))
testOp = testOp.drop('Sales_Contract_No', axis=1)

---

In [50]:
testOp = testOp.join(target_enc_CB.transform(testOp['Codigo_Burocratico']).add_suffix('_target'))
testOp = testOp.drop('Codigo_Burocratico', axis=1)

---

In [51]:
testOp = testOp.join(target_enc_AE.transform(testOp['Aprobacion_Especial']).add_suffix('_target'))
testOp = testOp.drop('Aprobacion_Especial', axis=1)

# Modelos

Random Forest

In [54]:
from sklearn.ensemble import RandomForestClassifier

In [55]:
parameters = {'n_estimators':[50, 100, 150, 200, 250, 300]}

In [56]:
modeloRFC=RandomForestClassifier(random_state=0, n_jobs=-1)

In [57]:
LogLoss = make_scorer(log_loss)

In [58]:
gscv = GridSearchCV(modeloRFC, parameters, cv=5, scoring=LogLoss)

In [59]:
gscv.fit(train_data, train_label)

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators='warn', n_jobs=-1,
                                              oob_score=False, random_state=0,
                                              verbose=0, warm_start=False),
             iid='warn', n_jobs=None,
             param_grid={'n_e

In [60]:
print(gscv.best_score_, gscv.best_estimator_)

1.5101868651838977 RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=-1,
                       oob_score=False, random_state=0, verbose=0,
                       warm_start=False)


In [61]:
modeloRFC=RandomForestClassifier(random_state=0, n_jobs=-1, n_estimators=50)

In [62]:
modeloRFC.fit(train_data, train_label)
result = modeloRFC.predict(testOp)

In [63]:
entregable_RFC = IDs[['Opportunity_ID']].copy()
entregable_RFC['Target'] = result

In [64]:
entregable_RFC.to_csv('RF_TodosLosFeatures.csv', index=False)